In [46]:
pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [47]:
pip install plotly

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [48]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [49]:
pip install seaborn

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [50]:
pip install dash

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [51]:
# Import standard libraries
import os
from contextlib import redirect_stdout

import sys
# append coeqwal packages to path
sys.path.append('./coeqwalpackage')

import numpy as np
import pandas as pd
import datetime as dt
import re
import plotly.graph_objects as go
import dash
from dash import html, dcc
from dash.dependencies import Input, Output

In [52]:
# Import custom libraries
# Note: on my computer the next import doesn't work the first time I call it, why? If I re-run the cell, then it is ok. MUST DEBUG
from coeqwalpackage.metrics import *
import cqwlutils as cu
import plotting as pu

In [53]:
CtrlFile = 'CalSim3DataExtractionInitFile_v4.xlsx'
CtrlTab = 'Init'
ScenarioListFile, ScenarioListTab, ScenarioListPath, DVDssNamesOutPath, SVDssNamesOutPath, ScenarioIndicesOutPath, DssDirsOutPath, VarListPath, VarListFile, VarListTab, VarOutPath, DataOutPath, ConvertDataOutPath, ExtractionSubPath, DemandDeliverySubPath, ModelSubPath, GroupDataDirPath, ScenarioDir, DVDssMin, DVDssMax, SVDssMin, SVDssMax, NameMin, NameMax, DirMin, DirMax, IndexMin, IndexMax, StartMin, StartMax, EndMin, EndMax, VarMin, VarMax, DemandFilePath, DemandFileName, DemandFileTab, DemMin, DemMax, InflowOutSubPath, InflowFilePath, InflowFileName, InflowFileTab, InflowMin, InflowMax = cu.read_init_file(CtrlFile, CtrlTab)

In [54]:
df, dss_names = read_in_df(ConvertDataOutPath,DVDssNamesOutPath)

In [55]:
df = add_water_year_column(df)

/Users/rain/Desktop/DSP/coeqwal/notebooks/coeqwalpackage/metrics.py:175: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  return df_copy.drop(["Date", "Year", "Month"], axis=1)


In [56]:
metrics_path = GroupDataDirPath + "/metrics_output"
if not os.path.exists(metrics_path):
    os.makedirs(metrics_path)

plots_path = GroupDataDirPath + "/plots_output"
if not os.path.exists(plots_path):
    os.makedirs(plots_path)
    

In [57]:
drought_wys = [
    1924,1925,1926,1929,1930,1931,1932,1933,1934,
    1939,1944,1945,1947,1948,1949,1950,1955,1960,
    1961,1962,1964,1976,1977,1979,1981,1987,1988,
    1989,1990,1991,1992,1994,2001,2008,2009,2013,
    2014,2015,2020,2021
]

In [58]:
def get_scenario_styles(studies):
    """
    Given a list (or tuple) of scenario numbers, return an appropriate dictionary
    specifying line color, style, and label. Extend as needed for more scenarios.
    """
    scenario_tuple = tuple(studies)
    if scenario_tuple == (2, 11):
        return {
            "s0002": {'color': 'black', 'linestyle': 'solid', 'label': 's0002 Baseline'},
            "s0011": {'color': 'red', 'linestyle': 'dash', 'label': 's0011 TUCP'}
        }
    elif scenario_tuple == (11, 12):
        return {
            "s0011": {'color': 'black', 'linestyle': 'solid', 'label': 's0011 Baseline (TUCP)'},
            "s0012": {'color': 'red', 'linestyle': 'dash', 'label': 's0012 SGMA'}
        }
    elif scenario_tuple == (11, 13):
        return {
            "s0011": {'color': 'black', 'linestyle': 'solid', 'label': 's0011 Baseline (TUCP)'},
            "s0013": {'color': 'red', 'linestyle': 'dot', 'label': 's0013 Future Baseline'}
        }
    else:
        colors = ['black', 'red', 'blue', 'green', 'orange']
        linestyles = ['solid', 'dash', 'dot', 'longdash', 'dashdot']
        style_dict = {}
        for i, s in enumerate(studies):
            style_dict[s] = {
                'color': colors[i % len(colors)],
                'linestyle': linestyles[i % len(linestyles)],  # Use valid Plotly styles
                'label': s  # Fix: Remove extra 's' prefix
            }
        return style_dict


In [59]:
variables = [
    "NOD_STORAGE_", "DEL_NOD_AG_", "DEL_NOD_MI_",
    "C_SAC041_", "C_SJR070_", "NDO_",
    "X2_PRV_KM_", "TOTAL_EXPORTS_",
    "DEL_SOD_AG_", "SOD_STORAGE_", "DEL_SOD_MI_"
]

scenario_comps = [[2,11],[11,12],[11,13]]

In [60]:
df.columns = ['_'.join(map(str, col)) for col in df.columns]

print(df.columns.tolist()[:20]) 


['CALSIM_COREQSACDV_s0001_FLOW_1MON_L2020A_PER-AVER_CFS', 'CALSIM_CO_EC_MONTH_s0001_SALINITY_1MON_L2020A_PER-AVER_UMHOS/CM', 'CALSIM_C_AMR004_s0001_CHANNEL_1MON_L2020A_PER-AVER_CFS', 'CALSIM_C_AMR004_ADD_s0001_FLOW-ADDITIONAL-INSTREAM_1MON_L2020A_PER-AVER_CFS', 'CALSIM_C_AMR004_MIF_s0001_FLOW-MIN-INSTREAM_1MON_L2020A_PER-AVER_CFS', 'CALSIM_C_CAA003_s0001_CHANNEL_1MON_L2020A_PER-AVER_CFS', 'CALSIM_C_CAA003_CVP_s0001_FLOW-DELIVERY_1MON_L2020A_PER-AVER_CFS', 'CALSIM_C_CAA003_SWP_s0001_FLOW-DELIVERY_1MON_L2020A_PER-AVER_CFS', 'CALSIM_C_CAA003_TD_s0001_FLOW-DELIVERY_1MON_L2020A_PER-AVER_CFS', 'CALSIM_C_CHW017_s0001_CHANNEL_1MON_L2020A_PER-AVER_CFS', 'CALSIM_C_CLV004_s0001_CHANNEL_1MON_L2020A_PER-AVER_CFS', 'CALSIM_C_DMC000_s0001_CHANNEL_1MON_L2020A_PER-AVER_CFS', 'CALSIM_C_DMC000_TD_s0001_FLOW-DELIVERY_1MON_L2020A_PER-AVER_CFS', 'CALSIM_C_DMC003_s0001_CHANNEL_1MON_L2020A_PER-AVER_CFS', 'CALSIM_C_EBP016_s0001_CHANNEL_1MON_L2020A_PER-AVER_CFS', 'CALSIM_C_FOLSM_s0001_CHANNEL_1MON_L2020A_PER-AV

In [61]:
original_columns = df.columns.tolist()

s_numbers = set()
for col in original_columns:
    matches = re.findall(r's\d{4,}', col)  
    s_numbers.update(matches) 

# Convert to a sorted list
s_numbers_list = sorted(s_numbers)

print(s_numbers_list)

['s0001', 's0002', 's0003', 's0004', 's0005', 's0006', 's0007', 's0008', 's0009', 's0010', 's0011', 's0012', 's0013', 's0014', 's0015', 's0016', 's0018']


## Interactive Dashboard

### Time Series
Shows how the selected variable changes over time for each scenario.

### Monthly-of-Year
Displays the monthly average for a selected year.

### Single Exceedance
Shows the probability that a value will be equaled or exceeded.

### Annual Exceedance
Shows how often the selected month's total value exceeds a given threshold across all years. Each year’s data for the chosen month is summed (e.g., total flow in April each year), and the annual values are ranked from highest to lowest. From these ranks, exceedance probabilities are calculated to show how frequently high values occur.

### Month-of-Year Avg
Averages each calendar month across all years, optionally filtered by Water Year Type. Water Year Types classify each year based on how wet or dry it was. The scale ranges from 1 (wettest) to 5 (driest)

In [62]:
def add_water_year_column(df):
    df_copy = df.copy().sort_index()
    df_copy['Date'] = pd.to_datetime(df_copy.index)
    df_copy['Year'] = df_copy['Date'].dt.year
    df_copy['Month'] = df_copy['Date'].dt.month
    df_copy['WaterYear'] = np.where(df_copy['Month'] >= 10, df_copy['Year'] + 1, df_copy['Year'])
    return df_copy.drop(["Date", "Year", "Month"], axis=1)

water_year_df = add_water_year_column(df)

# Extract variable names from DataFrame columns
pattern = re.compile(r'CALSIM_([^_]+(?:_[^_]+)*)_s\d{4}')
variables = sorted(set(
    match.group(1)
    for col in df.columns
    for match in [pattern.search(col)]
    if match
))


scenarios = sorted(set(re.findall(r's\d{4,}', ' '.join(df.columns))))
default_scenario = [scenarios[0]] if scenarios else []

years_in_data = sorted(df.index.year.unique())
drought_years = {
    1924, 1925, 1926, 1929, 1930, 1931, 1932, 1933, 1934, 1939,
    1944, 1945, 1947, 1948, 1949, 1950, 1955, 1960, 1961, 1962, 1964,
    1976, 1977, 1979, 1981, 1987, 1988, 1989, 1990, 1991, 1992, 1994,
    2001, 2008, 2009, 2013, 2014, 2015, 2020, 2021
}

year_options = [{"label": f"{y} {'(Drought)' if y in drought_years else ''}", "value": y} for y in years_in_data]
water_year_type_options = [{"label": str(i), "value": i} for i in range(1, 6)]
month_options = [{"label": "April", "value": 4}, {"label": "September", "value": 9}]

def find_col(df, variable, scenario):
    matches = [col for col in df.columns if variable in col and scenario in col]
    return matches[0] if matches else None

def find_wyt_col(df, scenario):
    matches = [col for col in df.columns if f"CALSIM_WYT_SAC__{scenario}" in col and "WATERYEARTYPE" in col]
    return matches[0] if matches else None

def get_colors(scenarios):
    base = ["red", "blue", "green", "orange", "purple", "brown", "cyan", "magenta", "gray", "black"]
    return {s: base[i % len(base)] for i, s in enumerate(scenarios)}

def filter_by_wyt_annual(df_col, scenario, wyt_list, month=5):
    if not wyt_list:
        return df_col

    wyt_col = find_wyt_col(df, scenario)
    if wyt_col is None:
        return df_col

    working_df = df[[wyt_col]].copy()
    working_df['WaterYear'] = water_year_df['WaterYear']
    working_df['Month'] = df.index.month

    filtered = working_df[working_df['Month'] == month].groupby('WaterYear').first()
    selected_years = filtered[filtered[wyt_col].isin(wyt_list)].index

    df_col = df_col.copy()
    df_col['WaterYear'] = water_year_df['WaterYear']
    df_col = df_col[df_col['WaterYear'].isin(selected_years)]
    return df_col.drop(columns='WaterYear')

def generate_time_series_plot(df, selected_variable, selected_scenarios):
    fig = go.Figure()
    scenario_colors = get_colors(selected_scenarios)

    for scenario in selected_scenarios:
        matching_cols = [col for col in df.columns if scenario in col and selected_variable in col]
        if not matching_cols:
            continue
        scenario_col = matching_cols[0]

        fig.add_trace(go.Scatter(
            x=df.index,
            y=df[scenario_col],
            mode='lines',
            name=scenario,
            line=dict(color=scenario_colors[scenario])
        ))

    fig.update_layout(
        title=f"Time Series Plot for {selected_variable}",
        xaxis_title="Date",
        yaxis_title="Value",
        hovermode="x"
    )
    return fig

def generate_month_of_year_avg_plot(df, var, scenarios, wyt_list):
    fig = go.Figure()
    colors = get_colors(scenarios)
    for s in scenarios:
        col = find_col(df, var, s)
        if col:
            df_copy = water_year_df[[col]].copy()
            df_copy = filter_by_wyt_annual(df_copy, s, wyt_list, month=5)
            df_copy['Month'] = df_copy.index.month
            monthly_avg = df_copy.groupby('Month')[col].mean()
            fig.add_trace(go.Scatter(x=monthly_avg.index, y=monthly_avg.values,
                                     mode='lines', name=s, line=dict(color=colors[s])))
    fig.update_layout(title=f"Month-of-Year Average for {var}",
                      xaxis_title="Month",
                      yaxis_title="Average Value")
    return fig

def generate_single_exceedance_plot(df, var, scenarios):
    fig = go.Figure()
    colors = get_colors(scenarios)
    for s in scenarios:
        col = find_col(df, var, s)
        if col:
            series = df[col].dropna().sort_values(ascending=False)
            exceedance_probs = np.arange(1, len(series)+1) / (len(series)+1)
            fig.add_trace(go.Scatter(x=exceedance_probs, y=series.values, mode="lines", name=s, line=dict(color=colors[s])))
    fig.update_layout(title=f"Single Exceedance Plot for {var}",
                      xaxis_title="Exceedance Probability",
                      yaxis_title="Value")
    return fig

def generate_annual_exceedance_plot(df, var, scenarios, month):
    fig = go.Figure()
    colors = get_colors(scenarios)
    for s in scenarios:
        col = find_col(df, var, s)
        if col:
            df_copy = df[[col]].copy()
            df_copy = df_copy[df_copy.index.month == month]
            annual_sum = df_copy.resample("YE").sum(min_count=1)
            sorted_vals = annual_sum[col].dropna().sort_values(ascending=False)
            exceed_probs = sorted_vals.rank(method="first", ascending=False) / (1 + len(sorted_vals))
            fig.add_trace(go.Scatter(x=exceed_probs, y=sorted_vals, mode="lines", name=s,
                                     line=dict(color=colors[s])))
    fig.update_layout(title=f"Annual Exceedance Plot for {var} (Month: {month})",
                      xaxis_title="Exceedance Probability",
                      yaxis_title="Annual Total")
    return fig

def generate_monthly_plot(df, var, scenarios, selected_year):
    fig = go.Figure()
    colors = get_colors(scenarios)

    for s in scenarios:
        col = find_col(df, var, s)
        if col:
            df_copy = df[[col]].copy()
            df_copy['Year'] = df_copy.index.year
            df_copy['Month'] = df_copy.index.month
            df_copy = df_copy[df_copy['Year'] == selected_year]

            monthly_avg = df_copy.groupby('Month')[col].mean()

            fig.add_trace(go.Scatter(
                x=monthly_avg.index,
                y=monthly_avg.values,
                mode='lines+markers',
                name=s,
                line=dict(color=colors[s])
            ))
    fig.update_layout(
        title=f"Monthly Plot for {var} in {selected_year}",
        xaxis_title="Month",
        yaxis_title="Average Value",
        xaxis=dict(tickmode='array', tickvals=list(range(1, 13)))
    )
    return fig

plot_type_options = [
    {"label": "Time Series", "value": "time_series"},
    {"label": "Monthly-of-Year", "value": "monthly"},
    {"label": "Single Exceedance", "value": "single_exceedance"},
    {"label": "Annual Exceedance", "value": "annual_exceedance"},
    {"label": "Month-of-Year Avg", "value": "month_of_year_avg"}
]

plot_type_descriptions = {
    "time_series": "Shows how the selected variable changes over time for each scenario.",
    "monthly": "Displays the monthly average for a selected year.",
    "single_exceedance": "Shows the probability that a value will be equaled or exceeded.",
    "annual_exceedance": (
        "Shows how often the selected month's total value exceeds a given threshold across all years. "
        "Each year’s data for the chosen month is summed (e.g., total flow in April each year), and the "
        "annual values are ranked from highest to lowest. From these ranks, exceedance probabilities are "
        "calculated to show how frequently high values occur."
    ),
    "month_of_year_avg": (
        "Averages each calendar month across all years, optionally filtered by Water Year Type. "
        "Water Year Types classify each year based on how wet or dry it was. The scale ranges from 1 (wettest) to 5 (driest)."
    )
}

app = dash.Dash(__name__)
app.layout = html.Div([
    html.H2("Water Data Dashboard", style={'textAlign': 'center'}),

    html.Label("Select Variable"),
    dcc.Dropdown(id="variable-dropdown", options=[{"label": v, "value": v} for v in variables], value=variables[0]),

    html.Label("Select Scenarios"),
    dcc.Dropdown(id="scenario-dropdown", options=[{"label": s, "value": s} for s in scenarios], value=default_scenario, multi=True),

    html.Label("Select Plot Type"),
    dcc.Dropdown(id="plot-type-dropdown", options=plot_type_options, value="time_series"),

    html.Div(id="plot-type-description", style={"marginTop": "10px", "fontStyle": "italic", "color": "#555"}),

    html.Div([
        html.Label("Select Year (for Monthly Plot)"),
        dcc.Dropdown(id="year-dropdown", options=year_options, value=years_in_data[0])
    ], id="year-container"),

    html.Div([
        html.Label("Select Water Year Type (1-5)"),
        dcc.Dropdown(id="wyt-dropdown", options=water_year_type_options, value=[], multi=True)
    ], id="wyt-container", style={"display": "none"}),

    html.Div([
        html.Label("Select Month for Annual Exceedance"),
        dcc.Dropdown(id="month-dropdown", options=month_options, value=9)
    ], id="month-container", style={"display": "none"}),

    dcc.Graph(id="dynamic-plot")
])

@app.callback(
    [Output("dynamic-plot", "figure"),
     Output("year-container", "style"),
     Output("wyt-container", "style"),
     Output("month-container", "style"),
     Output("plot-type-description", "children")],
    [Input("variable-dropdown", "value"),
     Input("scenario-dropdown", "value"),
     Input("plot-type-dropdown", "value"),
     Input("year-dropdown", "value"),
     Input("wyt-dropdown", "value"),
     Input("month-dropdown", "value")]
)
def update_plot(var, scenarios, plot_type, year, wyt, month):
    show_year = {"display": "block"} if plot_type == "monthly" else {"display": "none"}
    show_wyt = {"display": "block"} if plot_type == "month_of_year_avg" else {"display": "none"}
    show_month = {"display": "block"} if plot_type == "annual_exceedance" else {"display": "none"}

    description = plot_type_descriptions.get(plot_type, "")

    if not scenarios:
        return go.Figure(), show_year, show_wyt, show_month, description

    if plot_type == "time_series":
        fig = generate_time_series_plot(df, var, scenarios)
    elif plot_type == "monthly":
        fig = generate_monthly_plot(df, var, scenarios, year)
    elif plot_type == "single_exceedance":
        fig = generate_single_exceedance_plot(df, var, scenarios)
    elif plot_type == "annual_exceedance":
        fig = generate_annual_exceedance_plot(water_year_df, var, scenarios, month)
    elif plot_type == "month_of_year_avg":
        fig = generate_month_of_year_avg_plot(df, var, scenarios, wyt)
    else:
        fig = go.Figure()

    return fig, show_year, show_wyt, show_month, description


if __name__ == '__main__':
    app.run(debug=True)